In [1]:
!pip install git+https://github.com/huggingface/transformers --quiet
!apt install libomp-dev --quiet
!python -m pip install --upgrade faiss faiss-gpu --quiet
!pip install datasets --quiet
!pip install gitpython

!nvidia-smi

import time as t
from transformers import RagRetriever, RagSequenceForGeneration, RagTokenizer
from google.colab import drive

drive.mount('/content/drive')

%cd drive/MyDrive/rag_ft
%ls -l

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Reading package lists...
Building dependency tree...
Reading state information...
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Tue Feb  9 23:19:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [2]:
!pip install -r rag/requirements.txt

In [3]:
!python rag/finetune_rag.py \
    --data_dir rag_manual_qa_finetuning \
    --output_dir output_ft \
    --model_name_or_path facebook/rag-sequence-nq \
    --model_type rag_sequence \
    --gpus 1 \
    --distributed_retriever pytorch \
    --index_name custom \
    --passages_path base/my_knowledge_dataset \
    --index_path base/my_knowledge_dataset_hnsw_index.faiss

loading configuration file https://huggingface.co/facebook/rag-sequence-nq/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9fa67b34fdf52a61471482e3eda08e43096db6b17f0335c373b6e1f87a60f1b0.8f43598fa4635e2d95535caf736c7a64609c82256ec2dff95c1c30bda72c980a
INFO:transformers.configuration_utils:loading configuration file https://huggingface.co/facebook/rag-sequence-nq/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9fa67b34fdf52a61471482e3eda08e43096db6b17f0335c373b6e1f87a60f1b0.8f43598fa4635e2d95535caf736c7a64609c82256ec2dff95c1c30bda72c980a
Model config RagConfig {
  "architectures": [
    "RagSequenceForGeneration"
  ],
  "bad_words_ids": [
    [
      0,
      0
    ]
  ],
  "dataset": "wiki_dpr",
  "dataset_split": "train",
  "do_deduplication": true,
  "do_marginalize": false,
  "doc_sep": " // ",
  "exclude_bos_score": false,
  "generator": {
    "_name_or_path": "",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "ac

In [4]:
!python rag/eval_rag.py \
    --model_name_or_path output_ft/hparams.pkl \
    --model_type rag_sequence \
    --evaluation_set rag_manual_qa_finetuning/test.source \
    --gold_data_path rag_manual_qa_finetuning/test.target \
    --predictions_path e2e_preds.txt \
    --eval_mode e2e \
    --gold_data_mode ans \
    --n_docs 5 \
    --print_predictions \
    --recalculate \

INFO:__main__:Evaluate the following checkpoints: ['output_ft/hparams.pkl']
INFO:__main__:***** Running evaluation for output_ft/hparams.pkl *****
INFO:__main__:  Batch size = 8
INFO:__main__:  Predictions will be stored under e2e_preds.txt
Traceback (most recent call last):
  File "rag/eval_rag.py", line 312, in <module>
    main(args)
  File "rag/eval_rag.py", line 286, in main
    retriever = RagRetriever.from_pretrained(checkpoint, **model_kwargs)
  File "/usr/local/lib/python3.6/dist-packages/transformers/models/rag/retrieval_rag.py", line 416, in from_pretrained
    config = kwargs.pop("config", None) or RagConfig.from_pretrained(retriever_name_or_path, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/transformers/configuration_utils.py", line 376, in from_pretrained
    config_dict, kwargs = cls.get_config_dict(pretrained_model_name_or_path, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/transformers/configuration_utils.py", line 427, in get_config_dict
    con

In [ ]:
# Deprec: Fine tuning without your own dataset
!python rag/finetune_rag.py \
    --data_dir rag_manual_qa_finetuning \
    --output_dir output_ft \
    --model_name_or_path facebook/rag-sequence-nq \
    --model_type rag_sequence \
    --fp16 \
    --gpus 1